In [1]:
import IPython
import pickle, h5py, sys
import numpy as np
import networkx as nx
import pyemma as pe
import analysis_helper as ahelper
np.set_printoptions(precision=2)

/home/boltzmann/apps/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [14]:
# The most important file for WESTPA simulations is the master WESTPA h5 file. 
h = h5py.File('pre_prepped_results/west.h5')
print(", ".join(h.keys()))

bin_topologies, ibstates, iterations, summary, tstates


In [ ]:
# 1) bin_topologies contain the mappers used for the simulation, it's possible to pull the mapper object out of the file 
# 2) ibstates contain information regarding the basis (and initial) state information
# 3) iterations contain all the information on a per iteration basis
# 4) summary gives you a nice overall summary of the simulation including the # of walkers per iteration and how much time each 
#    iteration took
# 5) tstates contains information regarding target states

In [18]:
# Let's check out what's contained in the summary section
h['summary'][10:20]

array([( 80, 1., 5.56e-04, 0.32, 5.56e-05, 0.05,  9.47, 1.66, ''),
       (100, 1., 1.03e-02, 0.37, 1.03e-03, 0.05, 11.43, 2.45, ''),
       (100, 1., 5.55e-03, 0.19, 5.02e-04, 0.04, 11.39, 2.43, ''),
       (100, 1., 1.45e-02, 0.17, 1.25e-03, 0.04, 11.69, 2.51, ''),
       (100, 1., 1.48e-03, 0.19, 1.32e-03, 0.03, 11.72, 2.34, ''),
       (110, 1., 1.55e-02, 0.19, 1.55e-03, 0.03, 12.79, 2.54, ''),
       (130, 1., 1.55e-03, 0.27, 1.55e-04, 0.04, 15.08, 2.73, ''),
       (120, 1., 1.55e-04, 0.23, 1.55e-05, 0.03, 14.11, 2.62, ''),
       (140, 1., 1.55e-05, 0.15, 1.55e-06, 0.03, 16.99, 2.83, ''),
       (150, 1., 1.55e-06, 0.17, 1.55e-07, 0.03, 18.08, 2.97, '')],
      dtype=[('n_particles', '<i8'), ('norm', '<f8'), ('min_bin_prob', '<f8'), ('max_bin_prob', '<f8'), ('min_seg_prob', '<f8'), ('max_seg_prob', '<f8'), ('cputime', '<f8'), ('walltime', '<f8'), ('binhash', 'S64')])

In [19]:
# Let's see what we have in each iteration
print(", ".join(h['iterations/iter_%08d'%100].keys()))

bin_target_counts, ibstates, pcoord, seg_index, wtgraph


In [34]:
# 1) bin_target_counts give you how many walkers per bin we have
# 2) ibstates is basically the same as before, for this iteration (important for recycling)
# 3) pcoord contains your progress coordinates for the entire iteration
# 4) seg_index includes information regarding the specific walker, like the history parent id or recycling status
# 5) wtgraph includes all parents, not just the history parent
# 6) additional data that can be passed to WESTPA will be stored here under 'auxdata'

In [32]:
# seg_index includes important information regarding the history parents of each walker (parent_id) and if a walker 
# is recycled or not (endpoint_type). 
print(h['iterations/iter_%08d'%10]['seg_index'][...].shape)
h['iterations/iter_%08d'%10]['seg_index'][10:20]

(70,)


array([(0.03, 22, 1, 16, 0.12, 0.13, 1, 2),
       (0.03,  0, 2, 17, 0.11, 0.14, 1, 2),
       (0.03,  8, 2, 19, 0.12, 0.13, 1, 2),
       (0.03, 29, 1, 21, 0.12, 0.12, 1, 2),
       (0.04, 63, 4, 22, 0.09, 0.1 , 1, 2),
       (0.03, 25, 1, 26, 0.1 , 0.11, 1, 2),
       (0.02, 62, 1, 27, 0.11, 0.12, 1, 2),
       (0.03, 21, 1, 28, 0.07, 0.08, 1, 2),
       (0.02, 60, 1, 29, 0.07, 0.08, 1, 2),
       (0.05, 27, 1, 30, 0.12, 0.12, 1, 2)],
      dtype=[('weight', '<f8'), ('parent_id', '<i8'), ('wtg_n_parents', '<u8'), ('wtg_offset', '<u8'), ('cputime', '<f8'), ('walltime', '<f8'), ('endpoint_type', 'u1'), ('status', 'u1')])

In [33]:
# wtgraph includes all parent information (not just the history parent)
print(h['iterations/iter_%08d'%10]['wtgraph'][...].shape)
h['iterations/iter_%08d'%10]['wtgraph'][10:20]

(87,)


array([58, 42, 19, 48, 15, 10, 22,  0, 67,  8])

In [35]:
# An important WESTPA tool is called w_assign which can be used to assign indices to each walker. It's highly flexible since
# you can pass a python function to determine the index of each walker. By default w_assign will use the bin mapper found
# in the master WESTPA h5 file. Also, unless a python function is given, w_assign will generally expect a rectilinear binning
# scheme unless it's using the one in the master WESTPA h5 file. 
a = h5py.File('pre_prepped_results/assign.h5')
print(", ".join(a.keys()))

assignments, bin_labels, labeled_populations, npts, nsegs, state_labels, state_map, statelabels, trajlabels


In [65]:
# let's explore some of these options.

# w_assign expects a set of states and bins to assign everything to. states are generally expected to be metastable states
# and bins can be any binning scheme you want. You can map multiple bins to a single state

# In this example file I used 4 states, each of which falls into a single bin. You can see the names I gave under 'state_labels'
print(a['state_labels'][...])
# I have 100 bins (100 voronoi centers in this case), you can see how each bin maps to each state under 'state_map'
print(a['state_map'][...], a['state_map'][...].shape)
# We can also see what each bin is supposed to be under 'bin_labels', here is the voronoi center for bin 10
print(a['bin_labels'][10])

['high_low' 'low_high' 'low_low' 'high_high']
(array([4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=uint16), (101,))
center=array([15., 27.,  0.,  1.,  0.,  0.,  1.,  0.], dtype=float32)


In [55]:
# The main thing we want is an assignment for each walker so that we can calculate some properties for each bin/voronoi center
# e.g. a transition matrix from center to center or the average value of something in a particular center. This can be found
# under 'assignments', shape is (# of iterations, maximum # of walkers, # of dimensions)
print(a['assignments'][...].shape)
print(a['assignments'][10])

(100, 780, 2)
[[ 35  11]
 [ 22  76]
 [ 72  33]
 ...
 [100 100]
 [100 100]
 [100 100]]


In [64]:
# We can find the label that's assigned to each point using 'statelabels'
print(a['statelabels'][...].shape)
print(a['statelabels'][10][0:5])
# We can also find a 'trajlabel', this is calculated such that it retains the last state it visited. e.g. say a walker went
# into state 1, left but is not considered in another state yet, it will retain the index 1. This can be seen here with 
# 0th walker in iteration 10, [10][0] is currently not in a state (4), but has last visited state (1).
print(a['trajlabels'][10][0:5])

(100, 780, 2)
[[4 4]
 [4 4]
 [4 4]
 [4 4]
 [4 4]]
[[1 1]
 [4 4]
 [4 4]
 [4 4]
 [4 4]]


In [66]:
# We can now use some of this information to analyze our data. 

# Let's calculate a transition matrix using the assignments we have. 
# TODO: Port the code here, make sure they can fall back to pre-calculated
tm = np.load('pre_prepped_results/tm.npy')
tm = tm[1:,:][:,1:]
print(tm)
# TODO: Write a plotting function to plot the matrix and show how symmetrizing changes things

[[5.28e-03 1.80e-04 9.16e-05 ... 0.00e+00 3.53e-05 2.46e-05]
 [1.85e-04 4.03e-03 4.35e-05 ... 0.00e+00 0.00e+00 9.14e-04]
 [2.47e-05 0.00e+00 2.81e-03 ... 1.75e-05 0.00e+00 0.00e+00]
 ...
 [0.00e+00 4.58e-05 2.00e-05 ... 7.62e-05 0.00e+00 2.71e-06]
 [6.27e-05 3.51e-06 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [8.30e-05 1.56e-03 0.00e+00 ... 2.43e-05 0.00e+00 1.92e-04]]


In [5]:
# We can make a markov chain over the voronoi bins, the transition matrix has to be symmetrized to fulfill equilibrium 
# conditions for future analysis
stm = ahelper.symmetrize(tm)
# Row normalize to turn this into a proper row stochastic transition matrix
ntm = ahelper.row_normalize(stm)
print(ntm)

[[1.18e-01 4.07e-03 1.29e-03 ... 0.00e+00 1.09e-03 1.20e-03]
 [3.36e-03 7.41e-02 4.00e-04 ... 4.22e-04 3.23e-05 2.28e-02]
 [1.33e-03 4.99e-04 6.45e-02 ... 4.30e-04 0.00e+00 0.00e+00]
 ...
 [0.00e+00 4.65e-03 3.80e-03 ... 1.54e-02 0.00e+00 2.74e-03]
 [1.09e-01 3.89e-03 0.00e+00 ... 0.00e+00 0.00e+00 0.00e+00]
 [4.74e-03 1.09e-01 0.00e+00 ... 1.19e-03 0.00e+00 1.69e-02]]


In [67]:
# Use pyEmma to do PCCA+ coarse graining, finding metastable states and a transition matrix between them
MSM = pe.msm.MSM(ntm, reversible=True)
pcca = MSM.pcca(4) # We know we have 4 states in this case, skipping timescales calculation here
p = pcca.coarse_grained_stationary_probability
ctm = pcca.coarse_grained_transition_matrix
# This is our coarse grained, 4x4 transition matrix
print(ctm)
# p is the stationary probability distribution as implied by our transition matrix
print(p)

[[0.62 0.01 0.2  0.17]
 [0.01 0.86 0.06 0.07]
 [0.06 0.03 0.9  0.01]
 [0.05 0.03 0.   0.92]]
[0.1  0.17 0.34 0.39]


In [69]:
# let's find out what our coarse grained states look like on average! 
mstable_assign = pcca.metastable_assignment
# using the bin labels (voronoi centers) and finding the average value for the centers
bin_labels = ahelper.get_bin_labels(a)

print("metastable state 0")
print(bin_labels[mstable_assign.T==0].mean(axis=0))
print("metastable state 1")
print(bin_labels[mstable_assign.T==1].mean(axis=0))
print("metastable state 2")
print(bin_labels[mstable_assign.T==2].mean(axis=0))
print("metastable state 3")
print(bin_labels[mstable_assign.T==3].mean(axis=0))

metastable state 0
[4.08 3.5  0.5  0.08 0.42 0.5  0.   0.5 ]
metastable state 1
[21.76 21.49  0.    1.    0.    0.    1.    0.  ]
metastable state 2
[21.84  4.64  0.    0.96  0.04  0.2   0.12  0.68]
metastable state 3
[ 4.92 20.72  0.2   0.08  0.72  0.    0.96  0.04]


In [ ]:
# TODO: Write a tool to plot network graphs, turn both the full matrix and the coarse matrix to graphs and plot both